# Test Different Input-Model Combinations 

In [ ]:
# import libs
import cv2
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import time
import torch
import torch.optim as optim
from matplotlib.image import imread
from torch import nn
from torch.utils import data
from torchvision import transforms

%matplotlib inline

sys.path.append(os.path.join(os.getcwd(), '..'))
import ai8x
import parse_qat_yaml

sys.path.append(os.path.join(os.getcwd(), '../models/'))

# comment out for folded models
#b2rgb = importlib.import_module('models.bayer2rgb')

# comment out for unfolded model
b2rgb = importlib.import_module('models.b2rgb')

In [ ]:
class Args:
    def __init__(self, act_mode_8bit):
        self.act_mode_8bit = act_mode_8bit
        self.truncate_testset = False

In [ ]:
act_mode_8bit = True # For evaluation mode, input/output range: -128, 127

test_batch_size = 1

args = Args(act_mode_8bit=act_mode_8bit)

# change checkpoint according to the model
checkpoint_path_b2rgb = '/home/ezgiyucel/repo/ai8x-training/logs/final/checkpoint_q8.pth.tar'

qat_yaml_file_used_in_training_b2rgb = '/home/ezgiyucel/repo/ai8x-training/policies/qat_policy.yaml'

ai_device = 85
round_avg = True

# cats_vs_dogs

In [ ]:
from datasets import cats_vs_dogs
test_model = importlib.import_module('models.ai85net-cd')
data_path = '/data_fast/'
checkpoint_path = '/home/ezgiyucel/repo/ai8x-training/logs/cats_vs_dogs_trained/qat_checkpoint_q8.pth.tar'
qat_yaml_file_used_in_training = '/home/ezgiyucel/repo/ai8x-training/policies/qat_policy.yaml'
_, test_set_inter = cats_vs_dogs.catsdogs_get_datasets((data_path, args), load_train=False, load_test=True, bayer=True, fold=False)
_, test_set = cats_vs_dogs.catsdogs_get_datasets((data_path, args), load_train=False, load_test=True, bayer=True)
_, test_set_original = cats_vs_dogs.catsdogs_get_datasets((data_path, args), load_train=False, load_test=True, bayer=False)
print(len(test_set))


# camvid

In [ ]:
from datasets import camvid
test_model = importlib.import_module('models.ai85net-unet')
data_path = '/data/raw/CamVid_All/'
checkpoint_path = '/home/ezgiyucel/repo/ai8x-synthesis/trained/ai85-camvid-unet-large-q.pth.tar'
qat_yaml_file_used_in_training = '/home/ezgiyucel/repo/ai8x-training/policies/qat_policy_camvid.yaml'

# cifar100

In [ ]:
from datasets import cifar100
test_model = importlib.import_module('models.ai85net-nas-cifar')
data_path = '/data_fast/'
checkpoint_path = '/home/ezgiyucel/repo/ai8x-synthesis/trained/ai85-cifar100-qat8-q.pth.tar'
qat_yaml_file_used_in_training = '/home/ezgiyucel/repo/ai8x-training/policies/qat_policy_late_cifar.yaml'
_, test_set_inter = cifar100.cifar100_get_datasets((data_path, args), load_train=False, load_test=True, bayer=True, fold=False)
_, test_set = cifar100.cifar100_get_datasets((data_path, args), load_train=False, load_test=True, bayer=True)
_, test_set_original = cifar100.cifar100_get_datasets((data_path, args), load_train=False, load_test=True, bayer=False)
print(len(test_set))


# imagenet

In [ ]:
from datasets import imagenet
test_model = importlib.import_module('models.ai87net-imagenet-effnetv2')
data_path = '/data_ssd/'
checkpoint_path = '/home/ezgiyucel/repo/ai8x-synthesis/trained/ai87-imagenet-effnet2-q.pth.tar'
qat_yaml_file_used_in_training = '/home/ezgiyucel/repo/ai8x-training/policies/qat_policy_imagenet.yaml'
_, test_set_inter = imagenet.imagenet_get_datasets((data_path, args), load_train=False, load_test=True, bayer=True, fold=False)
_, test_set = imagenet.imagenet_get_datasets((data_path, args), load_train=False, load_test=True, bayer=True)
_, test_set_original = imagenet.imagenet_get_datasets((data_path, args), load_train=False, load_test=True, bayer=False)
print(len(test_set))

In [ ]:
test_dataloader_inter = data.DataLoader(test_set_inter, batch_size=test_batch_size, shuffle=False)
test_dataloader = data.DataLoader(test_set, batch_size=test_batch_size, shuffle=False)
test_dataloader_original = data.DataLoader(test_set_original, batch_size=test_batch_size, shuffle=False)
print(len(test_dataloader))
print(len(test_dataloader.dataset))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

qat_policy_b2rgb = parse_qat_yaml.parse(qat_yaml_file_used_in_training_b2rgb)
qat_policy = parse_qat_yaml.parse(qat_yaml_file_used_in_training)

ai8x.set_device(device=ai_device, simulate=act_mode_8bit, round_avg=round_avg)

# comment out for folded models
#model_b2rgb = b2rgb.bayer2rgbnet().to(device)

# comment out for unfolded model
model_b2rgb = b2rgb.B2RGB().to(device)

Run one of the following models according to the dataset.

In [ ]:
model = test_model.AI85CatsDogsNet().to(device)

In [ ]:
model = test_model.AI85NASCifarNet(bias="--use-bias").to(device)

In [ ]:
model = test_model.AI85UNetLarge(bias="--use-bias").to(device)

In [ ]:
model = test_model.AI87ImageNetEfficientNetV2(bias="--use-bias").to(device)

In [ ]:
# fuse the BN parameters into conv layers before Quantization Aware Training (QAT)
ai8x.fuse_bn_layers(model_b2rgb)
ai8x.fuse_bn_layers(model)

# switch model from unquantized to quantized for QAT
ai8x.initiate_qat(model_b2rgb, qat_policy_b2rgb)
ai8x.initiate_qat(model, qat_policy)

checkpoint_b2rgb = torch.load(checkpoint_path_b2rgb,map_location = device)
checkpoint = torch.load(checkpoint_path,map_location = device)

model_b2rgb.load_state_dict(checkpoint_b2rgb['state_dict'])
model.load_state_dict(checkpoint['state_dict'])

ai8x.update_model(model_b2rgb)
model_b2rgb = model_b2rgb.to(device)
ai8x.update_model(model)
model = model.to(device)

# bayer-to-rgb for CamVid

In [ ]:
def fold_data(img, fold_ratio=2):
    img_folded = None
    for i in range(fold_ratio):
        for j in range(fold_ratio):
            img_subsample = img[i::fold_ratio, j::fold_ratio, :]
            if img_folded is not None:
                img_folded = np.concatenate((img_folded, img_subsample), axis=2)
            else:
                img_folded = img_subsample
                
    return img_folded
def Bayer_filter(img):
    out =  np.zeros(shape=[img.shape[0],img.shape[1],1],dtype=img.dtype)
    for j in range(1,img.shape[1],2):
        for i in range(0,img.shape[0],2):
            out[i,j] = img[i, j, 2]
    for j in range(0,img.shape[1],2):
        for i in range(0,img.shape[0],2):
            out[i,j] = img[i, j, 1]
    for j in range(1,img.shape[1],2):
        for i in range(1,img.shape[0],2):
            out[i,j] = img[i, j, 1]
    for j in range(0,img.shape[1],2):
        for i in range(1,img.shape[0],2):
            out[i,j] = img[i, j, 0]
    return out

def Bayer_filter_2(img):
    out =  np.zeros(shape=[img.shape[0],img.shape[1],3],dtype=img.dtype)
    for j in range(1,img.shape[1],2):
        for i in range(0,img.shape[0],2):
            out[i,j,2] = img[i, j, 2]
            out[i,j,1] = 0
            out[i,j,0] = 0
    for j in range(0,img.shape[1],2):
        for i in range(0,img.shape[0],2):
            out[i,j,1] = img[i, j, 1]
            out[i,j,2] = 0
            out[i,j,0] = 0
    for j in range(1,img.shape[1],2):
        for i in range(1,img.shape[0],2):
            out[i,j,1] = img[i, j, 1]
            out[i,j,2] = 0
            out[i,j,0] = 0
    for j in range(0,img.shape[1],2):
        for i in range(1,img.shape[0],2):
            out[i,j,0] = img[i, j, 0]
            out[i,j,1] = 0
            out[i,j,2] = 0
    return out 
    
folder = "/data/raw/CamVid_All/CamVid/test orig/"
for filename in os.listdir(folder):
    sample_inp = cv2.imread(os.path.join(folder,filename))
    sample_inp = cv2.cvtColor(sample_inp, cv2.COLOR_BGR2RGB)
    
    # comment out for bayer inputs
    sample_inp_bayer = Bayer_filter_2(sample_inp)
    
    # comment out for folded inputs
    #sample_inp_bayer = Bayer_filter(sample_inp)
    #sample_inp_bayer = fold_data(sample_inp_bayer)
    
    sample_inp_bayer_folded = (torch.Tensor(sample_inp_bayer).permute([2,0,1]).unsqueeze(0) - 128.).to(device)
    model_b2rgb.eval()
    with torch.no_grad():
        l1_out = model_b2rgb(sample_inp_bayer_folded)
        l1_out_np = (l1_out[0].cpu().detach().numpy().transpose(1,2,0)+128)
        l1_out_np = l1_out_np.astype(np.uint8)
        from PIL import Image
        im = Image.fromarray(l1_out_np)
        im.save("/data/raw/CamVid_All/CamVid/test b2rgb/"+filename)

# Bilinear Interpolation for CamVid

In [ ]:
def Bayer_filter(img):
    out =  np.zeros(shape=[img.shape[0],img.shape[1],1],dtype=img.dtype)
    for j in range(1,img.shape[1],2):
        for i in range(0,img.shape[0],2):
            out[i,j] = img[i, j, 2]
    for j in range(0,img.shape[1],2):
        for i in range(0,img.shape[0],2):
            out[i,j] = img[i, j, 1]
    for j in range(1,img.shape[1],2):
        for i in range(1,img.shape[0],2):
            out[i,j] = img[i, j, 1]
    for j in range(0,img.shape[1],2):
        for i in range(1,img.shape[0],2):
            out[i,j] = img[i, j, 0]
    return out

folder = "/data/raw/CamVid_All/CamVid/test orig/"
for filename in os.listdir(folder):
    sample_inp = cv2.imread(os.path.join(folder,filename))
    sample_inp = cv2.cvtColor(sample_inp, cv2.COLOR_BGR2RGB)
    sample_inp_bayer = Bayer_filter(sample_inp)
    img = cv2.cvtColor(sample_inp_bayer,cv2.COLOR_BayerGR2RGB)
    from PIL import Image
    im = Image.fromarray(img)
    im.save("/data/raw/CamVid_All/CamVid/test inter/"+filename)

# Bayer-to-RGB + Model

In [ ]:
model_b2rgb.eval()
model.eval()
correct = 0
with torch.no_grad():
    for image, label in test_dataloader:
        image = image.to(device)
        
        #input_a = (image[0].cpu().detach().numpy().transpose([1, 2, 0])+128).astype(np.uint8)
        #plt.figure()
        #plt.imshow(input_a)
        
        primer_out = model_b2rgb(image)
        
        #result_1 = (primer_out[0].cpu().detach().numpy().transpose([1, 2, 0])+128).astype(np.uint8)
        #plt.figure()
        #plt.imshow(result_1)
        
        model_out = model(primer_out)
        result = np.argmax(model_out.cpu())
        if(label == result):
            correct = correct + 1 
print("accuracy:")
print(correct / len(test_set))

# Model

In [ ]:
model.eval()
correct = 0
with torch.no_grad():
    for image, label in test_dataloader_original:
        image = image.to(device)
        model_out = model(image)
        result = np.argmax(model_out.cpu())
        if(label == result):
            correct = correct + 1
print("accuracy:")
print(correct / len(test_set))

# Bilinear Interpolation + Model

In [ ]:
model.eval()
correct = 0
with torch.no_grad():
    for image, label in test_dataloader_inter:
        image = image.to(device)
        img = (128+(image[0].cpu().detach().numpy().transpose(1,2,0))).astype(np.uint8)
        img = cv2.cvtColor(img,cv2.COLOR_BayerGR2RGB)
        out_tensor = torch.Tensor(((img.transpose(2,0,1).astype(np.float32))/128-1)).to(device)
        out_tensor = out_tensor.unsqueeze(0)
        model_out = model(out_tensor)
        result = np.argmax(model_out.cpu())
        if(label == result):
            correct = correct + 1
print("accuracy:")
print(correct / len(test_set))
